SQL Server - what's changed between 2008 and 2019?

To use this notebook, select "Add New Connection" in the Attach To dropdown. Connect to a SQL Server 2019 test database that you are comfortable using as a playground.

2017

STRING\_AGG() can concatenate strings from a SELECT statement! You can add a separator. NULLs will not show up or add a separator. The string to be concatenated can be as complex as you like. To order the strings you need to add a WITHIN GROUP (ORDER BY \[column\] ASC) clause.

2016

STRING\_SPLIT does the opposite, splits a string and produces a table of values based on a supplied separator. It outputs a single column table where the column name is 'value'. You can alias this table and column the same as anything else.

In [ ]:
SELECT
      STRING_AGG(Val.Val1,',')
    , STRING_AGG(Val.Val1, '') /*If you want no separator*/
    , STRING_AGG(Val.Val1, 'some big separator')
FROM
    (
        VALUES
              ('a')
            , ('b')
            , ('c')
    ) AS Val(Val1)

In [ ]:
SELECT
      *
FROM STRING_SPLIT('a,b,c,d', ',')

SELECT
      *
FROM STRING_SPLIT('a,b,c,d', ',')/*This actually returns a table with a single column called value*/
WHERE [value] = 'a'


2016:

CREATE OR ALTER statements for Stored Procs, Views, Functions etc.

Also DROP IF EXISTS for all of these plus tables and schemas

In [ ]:
SELECT * FROM sys.procedures WHERE name = 'Test1'

In [ ]:
DROP PROCEDURE IF EXISTS dbo.Test1

In [ ]:
CREATE OR ALTER PROCEDURE dbo.Test1
AS
    SELECT 'Test'
GO

In [ ]:
EXEC dbo.Test1

In [ ]:
CREATE OR ALTER PROCEDURE dbo.Test1
AS
    SELECT 'Test1'
GO

EXEC dbo.Test1

2012:

Sequence objects. These are an alternative to IDENTITY columns. They are objects that generate an incrementing sequence of numbers. To get the next number use NEXT VALUE FOR [sequence object]. They are more flexible than IDENTITY columns, allow uniqueness across different columns/tables, and allow inserts into identity columns as standard. To use them for identity columns set the column default to NEXT VALUE FOR.

In [15]:
CREATE SEQUENCE dbo.TestSequence AS INT
START WITH 1 -- start sequence at 1
INCREMENT BY 1 -- increment by 1 each time
MINVALUE 1 -- value can never be lower than 1
MAXVALUE 100000 -- value can never be higher than 100000
CYCLE -- when next value is greater than 100000, cycle back to 0

In [23]:
SELECT NEXT VALUE FOR dbo.TestSequence

THROW can now be used to either raise an error or throw an error in the CATCH section of a TRY CATCH block

In [24]:
THROW
      50001 /* error number, made up by coder */
    , 'An error has occurred'
    , 1 /* state associated with the error */

In [30]:
BEGIN TRY
    RAISERROR ('An error message from inside the TRY block', 16, 1)
END TRY
BEGIN CATCH
    -- Something to handle/log error can go here e.g. ROLLBACK
    print 'error handled'; -- note, THROW needs the previous statement to terminate with a ';'
    THROW
    print 'if you reach this, you''ve gone too far' -- this will not be called as the THROW returns control to the caller
END CATCH

Data conversions have the TRY_CAST and TRY_Convert functions. These both return NULL if the conversion is not possible, instead of throwing an error.

In [13]:
SELECT
      TRY_CAST('01-jan-2020' AS DATE) AS ValidCast
    , TRY_CAST('41-Jan-2020' AS DATE) AS InvalidCast
    , TRY_CONVERT(DATETIME2, '01-Feb-2020', 101) AS ValidConvert
    , TRY_CONVERT(DATETIME2, '30-Feb-2020', 101) AS InvalidConvert

We also now have the PARSE and TRY_PARSE functions. These largely works like their CAST equivalents, but allow you to specify a region e.g. for money, and succeed more often

In [16]:
SELECT
      PARSE('Saturday 1 February 2020' AS DATETIME2 USING 'en-US') AS SuccessfulParse
    , TRY_PARSE('Blergh' AS DATETIME2 USING 'en-US') AS FailedParse
    , TRY_PARSE('Saturday 02/01/2020' AS DATETIME2 USING 'en-US') AS SuccessfulUSDateParse -- we switch to parsing the date in the US format
    , TRY_PARSE('$40.00' AS MONEY USING 'en-US') AS SuccessfulDollarParse
    , TRY_PARSE('£40.00' AS MONEY USING 'en-US') AS FailedPoundParse -- this fails because we are converting from American money but have a pound sign

CONCAT allows us to concatenate strings more easily. It automatically turns NULLs to empty strings, and converts to the most appropriate data type.

In [17]:
SELECT CONCAT('Hello', ' ', 'world'), CONCAT('Hello', '/', NULL, 'wo', 'r', NULL, 'ld', '.')

New date functions to make dates easier to manage. DATEFROMPARTS allows you to generate a date from component parts. There are similar functions for all other date/time data types. ALso EOMONTH returns the last day of the month for the date entered.

In [2]:
SELECT
      DATEFROMPARTS(2019, 4, 15) AS DateFromParts1 -- can generate a date from year, month, day
    , DATEFROMPARTS(YEAR(GETDATE()), MONTH(GETDATE()), 1) AS DateFromParts1stOfMonth -- e.g. 1st day of current month
    , EOMONTH(GETDATE()) AS EOMonthCurrentDate -- returns last day of month for date entered e.g. last day of current month
    , EOMONTH(GETDATE(), 3) AS EOMonth3MonthsLater -- Can add or subtract dates e.g. last day of month 3 months in future

CHOOSE and IIF as alternatives to CASE statements.

CHOOSE allows you to pick from a list. IIF returns A or B depending on if an expression is true or not.

In [ ]:
SELECT
      CHOOSE(3, 'this', 'is', 'the', 'array') AS CHOOSEExample -- 1st argument is the index position, starting at 1, all other arguments make up the array
    , IIF(1 > 2, 1, 2) AS IIFExample -- argument 2 is returned if argument 1 evaluates to true, otherwise argument 3 is returned

DECLARE @ExampleTable AS TABLE
    (
          ChooseExampleColumn INT NOT NULL
        , IifExampleColumn BIT NOT NULL
    )
INSERT INTO @ExampleTable
    (
          ChooseExampleColumn
        , IifExampleColumn
    )
VALUES
      (1, 'TRUE')
    , (2, 'FALSE')
    , (1, 'FALSE')
    , (5, 'TRUE')

SELECT -- you can use these with values in a column
      ChooseExampleColumn
    , CHOOSE(ChooseExampleColumn, 'value1', 'value2', 'value3', 'value4', 'value5', 'value6', 'value7') AS ChooseExample /* use the ChooseExampleColumn as the index */
    , IifExampleColumn
    , IIF(IifExampleColumn = 'TRUE', 'TRUE', 'NOT TRUE') AS IifExample
FROM @ExampleTable

Window functions now include a ROWS/RANGE clause. Both essentially add another restriction alongside the partition when making values available to the aggregate function.

ROWS specifies the number of rows ahead/behind the current row in the ORDER BY clause. It includes an UNBOUNDED option and a CURRENT ROW option. If nothing is specified the query assumes ROWS UNBOUND PRECEDING.

RANGE works the same as ROWS except it works on the ORDER BY position instead of the row. It cannot use N PRECEDING/FOLLOWING.

In [ ]:
DECLARE @ExampleTable AS TABLE
    (
          Col1 INT NOT NULL
        , PartitionColumn INT NOT NULL
    );
INSERT INTO @ExampleTable
    (
          Col1
        , PartitionColumn
    )
VALUES
      (1, 1)
    , (2, 1)
    , (3, 1)
    , (4, 1)
    , (5, 1)
    , (5, 1)
    , (1, 2)
    , (2, 2)
    , (3, 2);

SELECT
      Col1
    , PartitionColumn
    , SUM(Col1) OVER (PARTITION BY PartitionColumn ORDER BY Col1 ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    , SUM(Col1) OVER (PARTITION BY PartitionColumn ORDER BY Col1 ROWS UNBOUNDED PRECEDING) /* Assumes other side is CURRENT ROW */
    , SUM(Col1) OVER (PARTITION BY PartitionColumn ORDER BY Col1 RANGE UNBOUNDED PRECEDING)
    , SUM(Col1) OVER (PARTITION BY PartitionColumn ORDER BY Col1 ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING)
FROM @ExampleTable

FORMAT returns an inputted value formatted using a .NET Framework format string. It can also specify culture e.g. for date formatting.

In [ ]:
SELECT
      FORMAT(1234, '###-##.00')
    , FORMAT(1234, '0###-##.00')
    , FORMAT(CAST('01-jan-2020' AS DATE), 'D', 'en-gb')
    , FORMAT(CAST('01-jan-2020' AS DATE), 'D', 'en-us')
    , FORMAT(CAST('02-jan-2020' AS DATE), 'd', 'en-gb')
    , FORMAT(CAST('02-jan-2020' AS DATE), 'd', 'en-us')
    , FORMAT(25, 'C', 'en-us') /* Currency */
    , FORMAT(CAST('02-jan-2020' AS DATE), 'MM-yyyy-dd')
    , FORMAT(CAST('10:15' AS TIME), 'mm\.hh') /* . and : must be escaped in the format string */

LEAD/LAG windowed functions return the prevous/next values in the result set without needing a self join. You can define the offset, but the default is 1. You can define the value to be returned when the offset value does not exist, the default for that is NULL.

In [ ]:
DECLARE @ExampleTable AS TABLE
    (
          Col1 INT NOT NULL
        , PartitionColumn INT NOT NULL
    );
INSERT INTO @ExampleTable
    (
          Col1
        , PartitionColumn
    )
VALUES
      (1, 1)
    , (2, 1)
    , (3, 1)
    , (4, 1)
    , (5, 1)
    , (5, 1)
    , (1, 2)
    , (2, 2)
    , (3, 2);

SELECT
      Col1
    , PartitionColumn
    , LEAD(Col1) OVER(PARTITION BY PartitionColumn ORDER BY Col1)
    , LAG(Col1, 2, 0) OVER(PARTITION BY PartitionColumn ORDER BY Col1)
FROM @ExampleTable

PERCENTILE\_RANK/PERCENTILE\_CONT/PERCENTILE\_DISC/CUME\_DISC